# Exact Diagonalization of $J$-$K$ Hamiltonian
    Alan Morningstar
    May 2017

## Include Source

In [1]:
include("utils.jl");
include("lattice.jl");
include("basis.jl");
include("sparseHermitian.jl");
include("sparseHam.jl");
include("sparseS2.jl");

## Main

#### specify parameters

In [2]:
# square lattice length
const Lx = 4;
const Ly = 4;
# number of sites
const N = Lx*Ly;
# NN coupling
const J1 = 1.0;
# NNN coupling
const J2 = 0.0;
# plaquette coupling
const K = 0.0;

# choose Sz sector
const Sz = 0;
# choose kx,ky by specifying mi such that mi is in 0:Li-1
const mx = 2;
const my = 2;
const kx = 2*pi*mx/Lx;
const ky = 2*pi*my/Ly;
# choose spin inversion quantum number
const z = -1;

# number of eigenvalues desired
const numEigs = 4;
# a tolerance for error
const tolerance = 10.^(-12.);
# ritzVec = true if you want the eigenvectors returned too
const ritzVec = true;
# number of Krylov vectors in eigenvalue calculation
const numKrylovVecs = 60;
# maximum number of iterations to converge eigenvalues
const maxIter = 200;

# plaquette (x,y) vectors, locating p1,p2,p3,p4 on the plaquette of the p1 site and p1D,p2D,p1L,p3L on adjacent plaquettes
const neighborVectors = [(0,0),(1,0),(0,1),(1,1),(0,-1),(1,-1),(-1,0),(-1,1)];

#### define lattice, symmetry sector, and basis

In [3]:
# define the lattice
const l = lattice(Lx,Ly,neighborVectors);

In [4]:
# specify symmetry sector
const s = sector(Sz,kx,ky,z);

In [5]:
# construct the basis
# 0.34 seconds for 6x4 lattice, basis is ~0.002 GB
@time const basis = reducedBasis{UInt64}(l,s);
println("Dimension of reduced Hilbert space is ",basis.dim,".");

  0.177193 seconds (164.93 k allocations: 7.099 MB, 31.73% gc time)
Dimension of reduced Hilbert space is 386.


In [6]:
(sizeof(basis.b)+sizeof(basis.n))/10^9

6.176e-6

In [7]:
# using ProfileView;

In [8]:
# Profile.clear();
# @profile SzkxkyBasis(l,s);
# ProfileView.view()

#### build the Hamiltonian

In [9]:
# couplings type to make passing J1,K easier
const c = couplings(J1,J2,K);

# build the sparse Hamiltonian
# takes 0.95 seconds for 6x4 Heisenberg model, H contains ~0.013 GB
@time const H = constructSparseHam(basis,c,s,l);

  0.689419 seconds (655.22 k allocations: 23.262 MB, 0.80% gc time)


In [10]:
println("H: ",(sizeof(H.colPntrs)+sizeof(H.rowIndcs)+sizeof(H.nzVals)+sizeof(H.nzPntrs))/10^9," GB of memory.");

H: 3.1904e-5 GB of memory.


In [11]:
# Hamiltonian memory and time
# N=16 0.00015 GB, 0.012 s
# N=20 0.0020 GB, 0.15 s
# N=24 0.028 GB, 1.96 s
# N=30 1.6 GB
# N=32 ~6.0 GB, 8.4 m (extrapolated)
# N=36 ~96 GB, 2.2 h (extrapolated)

In [12]:
# using ProfileView;

In [13]:
# Profile.clear();
# @profile constructSparseHam(basis,c,s,l);
# ProfileView.view()

#### construct on the fly $H |\psi \rangle$ operator

In [14]:
# using LinearMaps;

In [15]:
# H_psi!(y::AbstractVector,x::AbstractVector) = H_mul_psi!(basis,c,s,l,y,x);

In [16]:
# HLM = LinearMap(H_psi!, basis.dim, Complex128, ismutating=true, ishermitian=true);

#### find eigenvalues and eigenvectors

In [17]:
# compute eigenvalues
# ~3.80 seconds for 6x4, numEigs=6, which=:SR, Heisenberg Hamiltonian
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);

  1.765660 seconds (2.18 M allocations: 85.715 MB, 1.35% gc time)


In [18]:
# using ProfileView;

In [19]:
# Profile.clear();
# @profile eigs(HLM; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);
# ProfileView.view()

In [23]:
# print energies
println("Energies are: ");
for en in real(eigsResult[1])
    println(en);
end;
# print algorithm performance
println("Number of iterations = ",eigsResult[3]);
println("Number of matrix-vector multiplications = ",eigsResult[4]);

Energies are: 
-10.649884872663453
-7.836537573479887
-7.3743978428830825
-6.632899657242488
Number of iterations = 4
Number of matrix-vector multiplications = 2


In [25]:
# # if we can check vs the dense diagonalization, do so
# if N <= 16
#     const Hdense = constructDenseHam(basis,c,s,l);
#     ens = sort!(real(eigvals(Hdense)));
#     println("Energies are: ");
#     for en in ens[1:2*numEigs]
#         println(en);
#     end;
# end;

In [26]:
# Lanczos time
# N=16 0.038 s
# N=20 0.53 s
# N=24 8.6 s
# N=30 
# N=32 ~37 m (extrap.)
# N=36 ~ 10 h (extrap.)

#### build the $S^2$ operator

In [27]:
# takes 3.60 seconds for 6x4 lattice, S2 contains ~0.066 GB
# @time const S2 = constructSparseS2(basis,s,l);

In [28]:
# println("S^2: ",(sizeof(S2.colPntrs)+sizeof(S2.rowIndcs)+sizeof(S2.nzVals)+sizeof(S2.nzPntrs))/10^9," GB of memory.");

In [29]:
# Total Spin memory
# N=16 0.00022 GB
# N=20 0.0038 GB
# N=24 0.066 GB
# N=30 4.7 GB
# N=32 ~18 GB (extrapolated)
# N=36 ~288 GB (extrapolated)

#### save spectral data
Save in format |  E  |  S(S+1)  |  Sz  |  mx  |  my  |

In [30]:
using DataFrames;

In [31]:
# energies
EData = real(eigsResult[1]);
# Sz values
SzData = fill(Sz,numEigs);
# mx values
mxData = fill(mx,numEigs);
# my values
myData = fill(my,numEigs);
# z values
zData = fill(z,numEigs);

In [36]:
# S(S+1) values
S2Data = zeros(Int64,numEigs);
psi = Array{Complex128,1}(basis.dim);
S2psi = Array{Complex128,1}(basis.dim);
@time for i in 1:numEigs
    psi = eigsResult[2][:,i];
    # A_mul_B!(S2psi,S2,psi);
    S2_mul_psi!(basis,s,l,S2psi,psi);
    S2Data[i] = round(Int64,real(dot(psi,S2psi))[1]);
end;

  0.026721 seconds (744 allocations: 71.741 KB)


In [37]:
# time to compute total spin of energy eigenstates
# N=16 0.061 s
# N=20 0.99 s
# N=24 20 s
# N=30 
# N=32 ~ 85 m (extrap.)
# N=36 ~ 23 h (extrap.)

In [38]:
# create DataFrame
df = DataFrame(E=EData,Ssqrd=S2Data,Sz=SzData,mx=mxData,my=myData);

In [39]:
println(df)

4×5 DataFrames.DataFrame
│ Row │ E        │ Ssqrd │ Sz │ mx │ my │
├─────┼──────────┼───────┼────┼────┼────┤
│ 1   │ -10.6499 │ 2     │ 0  │ 2  │ 2  │
│ 2   │ -7.83654 │ 12    │ 0  │ 2  │ 2  │
│ 3   │ -7.3744  │ 2     │ 0  │ 2  │ 2  │
│ 4   │ -6.6329  │ 2     │ 0  │ 2  │ 2  │
